In [5]:
import boto3
import botocore

In [6]:
def iterate_bucket_items(bucket):
    """
    Generator that iterates over all objects in a given s3 bucket

    See http://boto3.readthedocs.io/en/latest/reference/services/s3.html#S3.Client.list_objects_v2 
    for return data format
    :param bucket: name of s3 bucket
    :return: dict of metadata for an object
    """


    client = boto3.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket)

    for page in page_iterator:
        for item in page['Contents']:
            yield item

In [22]:



BUCKET_NAME = 'ts-dev-cs-training-data'
PREFIX = 'rbp-research/data/e2e/2017-09-11/'
KEY = 'rbp-research/data/e2e/2017-09-11/11fff4b9-1b6c-4e09-bb0e-809e882002bc.training.json'

s3 = boto3.resource('s3').Bucket(BUCKET_NAME)


def download_file(KEY, diskFilename):
    try:
        s3.download_file(KEY, diskFilename)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
            


In [8]:
itr = iterate_bucket_items(bucket=BUCKET_NAME)

print(next(itr))

{'Key': 'abbyy-hocr-backup/0000322e-16b6-4d36-b72c-5c9d827ba9dc.training', 'LastModified': datetime.datetime(2016, 7, 27, 20, 33, 11, tzinfo=tzutc()), 'ETag': '"e58c0f0c1d5bf873b4328490b5e5792c"', 'Size': 535661, 'StorageClass': 'STANDARD'}


In [3]:
download_file(KEY, 'test.json')

In [7]:
b.download_file()

In [20]:
def get_a_train_file_key(s3, prefix):
    for o in s3.objects.filter(Prefix=prefix, Delimiter='/'):
        yield o.key

In [23]:
gen = get_a_train_file_key(s3, PREFIX)

In [24]:
print(next(gen))

rbp-research/data/e2e/2017-09-11/00001593-256e-46f5-b0a1-bd0c09981c3a.training.json
